In [1]:
import numpy as np
import pandas as pd

In [2]:
sage_model = "/home/users/chege/theleap/ddecal/models/sagecal_format/rescaled_new3c61.sky.txt"
sage_clusters = "/home/users/chege/theleap/ddecal/models/sagecal_format/rescaled_new3c61.sky.txt.cluster"

In [3]:
def get_clusters_components(cluster_file):
    cluster_dict = {}
    with open(cluster_file) as cf:
        clusters = cf.readlines()
        for i, cl in enumerate(clusters):
            cl=cl.strip()
            if cl.strip() and not cl.startswith("#"):
                splitted=cl.strip("\n").split(" ")
                cluster_dict[splitted[0]] = {}
                cluster_dict[splitted[0]]['hf'] = splitted[1]
                cluster_dict[splitted[0]]['sources'] = splitted[2:]
    return cluster_dict

In [4]:
def sagecal_to_dp3(sage_model, sage_clusters, dp3_outfile):

    sage_cols = ['name', 'rah', 'ram', 'ras', 'decd', 'decm', 'decs', 'I', 'Q', 'U', 'V', 'spectral_index0', 'spectral_index1', 'spectral_index2', 'RotationMeasure', 'MajorAxis', 'MinorAxis', 'Orientation', 'ReferenceFrequency', 'empty']
    dp3_fields = ['Patch', 'rah', 'ram', 'ras', 'decd', 'decm', 'decs', 'I', 'Q', 'U', 'V', 'RotationMeasure', 'MajorAxis', 'MinorAxis', 'Orientation', 'ReferenceFrequency', 'SpectralIndex', 'LogarithmicSI']

    sage_df = pd.read_csv(sage_model,index_col=0, sep=" ", skiprows=5, names=sage_cols, comment='#').drop('empty', axis=1)

    # move 'spectral_index0', 'spectral_index1', 'spectral_index2' to a single 'SpectralIndex' column and drop them
    spec_indices = list(zip(sage_df['spectral_index0'], sage_df['spectral_index1'], sage_df['spectral_index2']))
    sage_df['SpectralIndex'] = [list (s) for s in spec_indices]
    sage_df = sage_df.drop(['spectral_index0', 'spectral_index1', 'spectral_index2'], axis=1)

    # add a LogarithmicSI column. Sagecal only supports this log spectral index format
    sage_df['LogarithmicSI'] = ["true"] * len(sage_df)

    di_model = get_clusters_components(sage_clusters)
    print(di_model.keys())


    def _get_dir_df(sage_df, direc, d):
        direc_name = f"cluster{d}"
        direction = di_model[direc]['sources']
        direc_df = sage_df[sage_df.index.isin(direction)]
        direc_df['Patch'] = [direc_name] * len(direc_df)
        direc_df = direc_df[dp3_fields]

        return direc_df


    directions = [_get_dir_df(sage_df, direc, d) for (d, direc) in enumerate(list(di_model.keys()))]


    with open(dp3_outfile, "w") as dm:
        format = "Format = Name," + ",".join(list(directions[0].columns))
        dm.writelines(f"{format}\n")

    with open(dp3_outfile, "a") as dm:
        for direc_df in directions:
            direc_title =  ['', direc_df["Patch"][0], "1", "17", "12.261", "89", "28", "48.916\n"]
            dm.writelines(f"{','.join(direc_title)}\n")

       
            direc_df.to_csv(dp3_outfile, sep=',', mode='a', header=False)
            

    # ncp_main = di_model['-1']['sources']
    # ncp_3c61 = di_model['-2']['sources']

    # ncp_main_df = sage_df[sage_df.index.isin(ncp_main)]
    # ncp_3c61_df = sage_df[sage_df.index.isin(ncp_3c61)]

    # ncp_main_df['Patch'] = ['MAIN_NCP_FIELD'] * len(ncp_main_df)
    # ncp_3c61_df['Patch'] = ['3C61_FIELD'] * len(ncp_3c61_df)


    # # Start writing out the dp3 model format
    # main_field_format =  ['', 'MAIN_NCP_FIELD', "1", "17", "12.261", "89", "28", "48.916\n"]
    # m3c61_format =  ['', '3C61_FIELD', "2", "22", "34.44", "86", "17", "7.89\n"]
    # print(len(ncp_main_df.keys()))

    # dp3_fields = ['Patch', 'rah', 'ram', 'ras', 'decd', 'decm', 'decs', 'I', 'Q', 'U', 'V', 'RotationMeasure', 'MajorAxis', 'MinorAxis', 'Orientation', 'ReferenceFrequency', 'SpectralIndex', 'LogarithmicSI']
    # ncp_main_df = ncp_main_df[dp3_fields]
    # ncp_3c61_df = ncp_3c61_df[dp3_fields]

    # print(len(ncp_main_df.keys()))

    # with open(dp3_outfile, "w") as dm:
    #     format = "Format = Name," + ",".join(list(ncp_main_df.columns))
    #     dm.writelines(f"{format}\n")

    # with open(dp3_outfile, "a") as dm:
    #     dm.writelines(f"{','.join(main_field_format)}\n")

    # ncp_main_df.to_csv(dp3_outfile, sep=',', mode='a', header=False)

    # with open(dp3_outfile, "a") as dm:
    #     dm.writelines(f"{','.join(m3c61_format)}\n")

    # ncp_3c61_df.to_csv(dp3_outfile, sep=',', mode='a', header=False)



In [5]:
sagecal_to_dp3(sage_model, sage_clusters, 'dp3out.txt')

dict_keys(['-1', '-2'])


/tmp/ipykernel_1292652/516712724.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direc_df['Patch'] = [direc_name] * len(direc_df)
/tmp/ipykernel_1292652/516712724.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direc_df['Patch'] = [direc_name] * len(direc_df)
/tmp/ipykernel_1292652/516712724.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

In [1]:
import lsmtool

In [12]:
di_model = lsmtool.load("/home/users/chege/theleap/twoleap/test/models/rescaled_new3c61.sky.bbs.txt")
di_model.plot("di_intrinsic_rescaled.pdf", labelBy='patch')

In [8]:
dd_model = lsmtool.load("/home/users/chege/theleap/twoleap/test/models/sky_sagecal_no_diffuse_simple_ateam_corrected_shape.bbs")
dd_model.plot("dd_intrinsic_rescaled.pdf")

In [9]:
ateam = lsmtool.load("/home/users/chege/theleap/nenuflow/models/Ateam_LBA.skymodel")

In [18]:
ateam.concatenate(di_model)

In [22]:
ateam.setPatchPositions(method='wmean')

In [24]:
ateam.write("ncp_and_ateam.txt", addHistory=True, clobber=True)